At this point the individual data cleaning pipelines for the four CSV files have been completed and can be found in `data_cleaning_pipelines.ipynb`.
<br><br>
The `products.csv`, `orders.csv` and `orderlines.csv` CSV files have been transformed and stored as `products_clean.csv`, `orders_clean.csv` and `orderlines_clean.csv`.
<br><br>
Many of the values in the `order_lines.unit_price`, `products.price` and `products.promo_price` values are corrupted and the correct values can only be determined by comparing the values across the tables.
`orders.total_paid` appears to be uncorrupted.
<br><br>
Here we will use test driven developement to create a pipeline to clean the values and then add the the pipeline to `data_cleaning_pipelines.ipynb`.

In [1]:
import re
import numpy as np
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import data_utils

## Import data

In [2]:
# Import the data
orders = data_utils.clean_orders(data_path="../../data/")
orderlines = data_utils.clean_orderlines(data_path="../../data/")
products = data_utils.clean_products(data_path="../../data/")
brands = data_utils.clean_brands(data_path="../../data/")

# Merge the data
completed_sales =  data_utils.merge_data(orders, orderlines, products, brands)

5 missing values were removed from orders.
This represents 0.0022% of the data.


0 missing values were removed from orderlines.
This represents 0.00% of the data.


8792 missing values were removed from products
This represents 45.49% of the data.


0 missing values were removed from brands.
This represents 0.00% of the data.




## Merge data

In [5]:
col_order = [
    'order_id',
    'orderline_id',
    'date',
    'name',
    'desc',
    'brand',
    'sku',
    'category',
    'total_paid',
    'product_quantity',
    'regular_price',
    'sale_price'
]

def reorder_columns(df, col_list):
    return df[col_list]

def start_pipeline(df):
    '''Make a copy of the pipeline to prevent corrupting the original data'''
    return df.copy()
    
def drop_deprecated_columns(df, col_list):
    return (df
            .drop(col_list, axis=1)
           )

def rename_columns(df, col_dict):
    return (df
            .rename(columns=col_dict)
           )
    
def assign_product_categories(df):
    apple_regexp_dict = {
        'iPod': '^.{0,7}apple ipod',
        'iPhone':  'apple iphone',
        'iPad':  'apple ipad',
        'Mac':  'apple macbook|apple iMac|apple Mac mini|desktop computer',
    }
    
    other_regexp_dict = {        
        'Smartwatch':'withings|watch|fitbit|apple watch|smartwatch|smart watch',
        'Accessories': 'kit|strap|armband|belt|bracelet|stylus|pen|Bamboo Wacom Intuos|pencil|pen|rubber pointers|screwdriver|case|funda|housing|casing|folder|bag|backpack|cable|connector|Lightning to USB|Wall socket|power strip|adapter|battery|headset|headphones|mouse|trackpad|stand|support|protect|cover|sleeve|Screensaver|shellhub|dock|microphone|keyboard|keypad',
        'Hardware': 'Philips Hue|temperature sensor|display|monitor|camera|charger|speaker|router|repeater|Synology|nas|server|Parrot FPV Glasses|Command Pack 2 Skycontroller|Apple TV',
        'Software':  'adobe|Office 365|Office Home and Student|software|parallels',
        'Memory': 'hard disk|hard drive|flash drive|USB 2.0 key|USB 2.0 pen|SSD|pendrive|raid|SDHC|sata|memory card|Portable Hard Thunderbolt',
        'Repairs & warranties': 'repair|parts and labor|warranty|applecare|license|protection|installation',
    }
    
    df = df.assign(category = 'unknown')
    
    # Find main apple items
    for label, val in apple_regexp_dict.items(): 
        regexp = re.compile(val, flags=re.IGNORECASE)
        df = (
            df
            .assign(
                category = lambda x: np.where(
                    ((x['desc'].str.contains(regexp, regex=True))|(x['name'].str.contains(regexp, regex=True))) &
                    (x['category'] == 'unknown') & (x['brand'] == 'Apple'), 
                    label, x['category'])
            )
        )
    
    # Find other items
    for label, val in other_regexp_dict.items(): 
        regexp = re.compile(val, flags=re.IGNORECASE)
        df = (
            df
            .assign(
                category = lambda x: np.where(
                    ((x['desc'].str.contains(regexp, regex=True))|(x['name'].str.contains(regexp, regex=True))) &
                    (x['category'] == 'unknown'), label, x['category'])
            )
        )
    
    return df

def merge_dataframes(df, merge_df, col):
    return df.merge(merge_df, on=col)

def drop_uncompleted_orders(df):
    return df[df.state=='Completed']

completed_sales = (orders
                   .pipe(start_pipeline)
                   .pipe(drop_uncompleted_orders)
                   .pipe(merge_dataframes, orderlines, 'order_id')
                   .pipe(merge_dataframes, products, 'sku')
                   .pipe(merge_dataframes, brands, 'short')
                   .pipe(rename_columns, col_dict={'long': 'brand', 'unit_price': 'sale_price', 'price': 'regular_price', 'id': 'orderline_id'})
                   .pipe(drop_deprecated_columns, col_list=['short', 'created_date', 'state'])
                   .pipe(assign_product_categories)
                   .pipe(reorder_columns, col_order)
             )

completed_sales.head()

,order_id,orderline_id,date,name,desc,brand,sku,category,total_paid,product_quantity,regular_price,sale_price
0,241423,1398738,2017-11-06 12:47:20,LaCie Porsche Design Desktop Drive 4TB USB 3.0...,External Hard Drive 4TB 35-inch USB 3.0 for Ma...,LaCie,LAC0212,Memory,136.15,1,139.99,129.16
1,242832,1529178,2017-12-31 17:26:40,Parrot 550mAh battery for MiniDrones,550mAh rechargeable battery for Parrot minidrones,Parrot,PAR0074,Accessories,15.76,1,17.99,10.77
2,243330,1181923,2017-02-15 17:07:44,Mac OWC Memory 8GB 1066MHZ DDR3 SO-DIMM,8GB RAM Mac mini iMac MacBook and MacBook Pro ...,OWC,OWC0074,unknown,84.98,1,99.99,77.99
3,245275,1276706,2017-06-28 11:12:30,Tado Smart Climate Control Intelligent AC,intelligent control air conditioning works wit...,Tado,TAD0007,Accessories,149.00,1,179,149.00
4,245595,1154394,2017-01-21 12:49:00,"Macally External Hard Drive 1TB 35 ""USB 3.0 SA...",Aluminum External Hard Drive 1TB capacity form...,Pack,PAC1561,Memory,112.97,2,103.95,52.99


# Data integrity tests
## How corrupted is the data?
Count the number of decimal points in each price value to identify corrupted values.

In [7]:
# Count the decimal points
prices = completed_sales[['orderline_id', 'total_paid', 'regular_price', 'sale_price']].copy()

prices['regular_price_decimal_count'] = prices['regular_price'].str.count(r'\.')
prices['sale_price_decimal_count'] = prices['sale_price'].str.count(r'\.')

prices

,orderline_id,total_paid,regular_price,sale_price,regular_price_decimal_count,sale_price_decimal_count
0,1398738,136.15,139.99,129.16,1,1
1,1529178,15.76,17.99,10.77,1,1
2,1181923,84.98,99.99,77.99,1,1
3,1276706,149.00,179,149.00,0,1
4,1154394,112.97,103.95,52.99,1,1
...,...,...,...,...,...,...
61667,1649446,18.98,35,13.99,0,1
61668,1649512,24.97,25,9.99,0,1
61669,1649522,24.97,25,9.99,0,1
61670,1649565,34.96,25,9.99,0,1


In [8]:
prices[['regular_price_decimal_count', 'sale_price_decimal_count']].apply(pd.Series.value_counts)

,regular_price_decimal_count,sale_price_decimal_count
0,18611,NaN
1,42264,58120.0
2,797,3552.0


## Create test data
Extract values with only one decimal so they can be transformed to floats and regular_price >= promo_price >= sale_price to create tests for cleaning the rest of the data.

In [9]:
# Extract a subset of the data with only one decimal point so we can transform the str values to floats and check our test logic.
test_data_ids = prices[
(prices.regular_price_decimal_count == 1) & 
(prices.sale_price_decimal_count == 1)].orderline_id
test_data = completed_sales[completed_sales.orderline_id.isin(test_data_ids)].copy()

# Transform the price values to float.
test_data.regular_price = test_data.regular_price.astype(float)
test_data.sale_price = test_data.sale_price.astype(float)

# This data is corrupted and should fail the tests
failing_test_data = test_data
# This subset of the data has regular_price >= sale_price and should therefore (hopefully) be uncorrupted
passing_test_data = test_data[test_data.regular_price >= test_data.sale_price]

print(failing_test_data.shape, passing_test_data.shape)

(41135, 12) (39858, 12)


In [10]:
''' Create functions which combine the logic above so they can be imported into other notebooks '''

def count_decimal_points(df):
    prices = df[['orderline_id', 'total_paid', 'regular_price', 'sale_price']].copy()

    prices['regular_price_decimal_count'] = prices['regular_price'].str.count(r'\.')
    prices['sale_price_decimal_count'] = prices['sale_price'].str.count(r'\.')

    return prices


def generate_test_data(sales_df):
    decimal_points_per_price = count_decimal_points(sales_df)
    
    # Extract a subset of the data with only one decimal point so we can transform the str values to floats and check our test logic.
    test_data_ids = decimal_points_per_price[
                        (decimal_points_per_price.regular_price_decimal_count == 1) & 
                        (decimal_points_per_price.sale_price_decimal_count == 1)
                    ].orderline_id
    test_data = sales_df[sales_df.orderline_id.isin(test_data_ids)].copy()

    # Transform the price values to float.
    test_data.regular_price = test_data.regular_price.astype(float)
    test_data.sale_price = test_data.sale_price.astype(float)
    
    # This data is corrupted and should fail the tests
    failing_test_data = test_data
    # This subset of the data has regular_price >= sale_price and should therefore (hopefully) be uncorrupted
    passing_test_data = test_data[test_data.regular_price >= test_data.sale_price]

    return failing_test_data, passing_test_data

failing_test_data, passing_test_data = generate_test_data(completed_sales)

print(failing_test_data.shape, passing_test_data.shape)

(41135, 12) (39858, 12)


## Tests
We will utilise tests that return data to help debug the logic for cleaning the price values.

### Test prices greater or equal

In [11]:
def test_col_vals_are_greater_or_equal_to_other(df, greater_col, lesser_col):
    num_incorrect_vals = df[df[greater_col] < df[lesser_col]].shape[0]
    if num_incorrect_vals == 0:
        print(f"All of the {greater_col} values are greater or equal to the corresponding {lesser_col} values.\n")
        return pd.DataFrame()
    else:
        print(f"There are corrupted values in {greater_col} which are less than their corresponding {lesser_col} values.")
        print(f"This respresents {num_incorrect_vals/df.shape[0]*100:.2f}% of the data.\n")
        corrupted_price_orderline_ids = df[df[greater_col] < df[lesser_col]].orderline_id
        return corrupted_price_orderline_ids

def test_regular_greater_or_equal_to_sale(df):
    incorrect_val_ids = test_col_vals_are_greater_or_equal_to_other(df, 'regular_price', 'sale_price')
    return incorrect_val_ids

In [13]:
regular_less_than_sale_orderline_ids = test_regular_greater_or_equal_to_sale(failing_test_data)

There are corrupted values in regular_price which are less than their corresponding sale_price values.
This respresents 3.10% of the data.



In [14]:
regular_less_than_sale_orderline_ids = test_regular_greater_or_equal_to_sale(passing_test_data)

All of the regular_price values are greater or equal to the corresponding sale_price values.



### Test total_paid

In [15]:
def test_order_id_has_single_total_paid_value(df):
    """
    A single order_id can have multiple lines corresponding to orderlines.
    Each of these lines should have the same total_paid value
    """
    
    # Group by order_id and check the number of unique total_paid values for each group
    inconsistent_orders = df.groupby('order_id')['total_paid'].nunique()
    
    # Filter for orders where there is more than 1 unique total_paid value
    inconsistent_orders = inconsistent_orders[inconsistent_orders > 1]
    
    # Check if there are any inconsistencies
    if not inconsistent_orders.empty:
        print("Inconsistent 'total_paid' values found for the following order_ids:")
        print(inconsistent_orders)
    else:
        print("All 'total_paid' values are consistent for each 'order_id'.")

test_order_id_has_single_total_paid_value(completed_sales)

All 'total_paid' values are consistent for each 'order_id'.


In [16]:
def test_order_total_paid_equal_sum_of_orderlines(df):
    """
    Group by order_id and calculate the sum of sale_price*product_quantity for all orderline_ids
    """
    grouped_orderlines = df.groupby('order_id').apply(
        lambda x: pd.Series({
            'calculated_total': (x['product_quantity'] * x['sale_price']).sum(),
            'total_paid': x['total_paid'].iloc[0]  # Total paid is the same for all rows in the group
        }),
        include_groups=False
    )

    # Compare calculated_total with total_paid
    incorrect_orders = grouped_orderlines[grouped_orderlines['calculated_total'] != grouped_orderlines['total_paid']]
    
    if incorrect_orders.empty:
        print("All orders have total_paid values equal to the sum of product_quantity * sale_price.")
        return pd.DataFrame()
    else:
        print(f"There are {incorrect_orders.shape[0]} orders where total_paid does not match the sum of product_quantity * sale_price.")
        print(f"This respresents {incorrect_orders.shape[0]/df.shape[0]*100:.2f}% of the data.\n")
        return incorrect_orders

In [17]:
incorrect_total_price = test_order_total_paid_equal_sum_of_orderlines(failing_test_data)

There are 26590 orders where total_paid does not match the sum of product_quantity * sale_price.
This respresents 64.64% of the data.



In [18]:
incorrect_total_price = test_order_total_paid_equal_sum_of_orderlines(passing_test_data)

There are 25956 orders where total_paid does not match the sum of product_quantity * sale_price.
This respresents 65.12% of the data.



<div class="alert alert-block alert-danger">
    Even when regular_price >= sale_price the sale price values do not match the total_paid per order.
    <br><br>
    It is possible that the difference in price is due to shipping costs, which are not included in the dataset.
    <br><br>
    Even though the 'passing' test data is clearly not as clean as we had hoped. We will leave as is for now so it can be used to debug why the prices do not match total_paid.
</div>